In [ ]:
!pip install transformers
!pip install torch

In [13]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch

# Load the dataset
file_path = '/kaggle/input/recommend-dishes-on-mood/recommended_dishes_large.csv'
dataset = pd.read_csv(file_path)

# Convert 'Weight' column to numerical values
weight_mapping = {'Light': 0, 'Moderate': 1, 'Heavy': 2}
dataset['Weight'] = dataset['Weight'].map(weight_mapping)

# Prepare training data with mood-based prompts
mood_map = ['happy', 'sad', 'stressed', 'relaxed']

training_data = []
for _, row in dataset.iterrows():
    natural_prompt = f"I want a dish for when I am feeling {mood_map[row['Mood']]}"
    output_text = f"Dish: {row['DishName']}"
    training_data.append({"input": natural_prompt, "output": output_text})

# Ensure the training data is diverse and balanced
# Optional: You can shuffle the dataset to ensure randomness
import random
random.shuffle(training_data)

# Tokenize the data
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
input_texts = [item['input'] for item in training_data]
output_texts = [item['output'] for item in training_data]

inputs = tokenizer(input_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
outputs = tokenizer(output_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Shift the labels for teacher forcing
labels = outputs["input_ids"]
labels[labels == tokenizer.pad_token_id] = -100  # Replace padding token id's with -100

# Create PyTorch dataset
class DishesDataset(torch.utils.data.Dataset):
    def __init__(self, input_encodings, labels):
        self.input_encodings = input_encodings
        self.labels = labels

    def __len__(self):
        return len(self.input_encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

dataset = DishesDataset(inputs, labels)

In [14]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=10,  # Increased epochs for better training
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Initialize Trainer
trainer = Trainer(
    model=BartForConditionalGeneration.from_pretrained("facebook/bart-base"),
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
trainer.save_model("/kaggle/working/fine-tuned-bart-dishes")
tokenizer.save_pretrained("/kaggle/working/fine-tuned-bart-dishes")

/tmp/ipykernel_34/3488644574.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
/tmp/ipykernel_34/3488644574.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])


Step,Training Loss
500,0.887400
1000,0.522600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/kaggle/working/fine-tuned-bart-dishes/tokenizer_config.json',
 '/kaggle/working/fine-tuned-bart-dishes/special_tokens_map.json',
 '/kaggle/working/fine-tuned-bart-dishes/vocab.json',
 '/kaggle/working/fine-tuned-bart-dishes/merges.txt',
 '/kaggle/working/fine-tuned-bart-dishes/added_tokens.json')

In [16]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained("/kaggle/working/fine-tuned-bart-dishes")
model = BartForConditionalGeneration.from_pretrained("/kaggle/working/fine-tuned-bart-dishes")

# Function to generate a recommendation from a mood-based prompt
def generate_recommendation(natural_prompt):
    input_ids = tokenizer.encode(natural_prompt, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=50, num_return_sequences=1)
    recommendation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return recommendation

# Example inputs
sample_inputs = [
    "I want a dish for when I am feeling happy",
    "I want a dish for when I am feeling sad",
    "I want a dish for when I am feeling stressed",
    "I want a dish for when I am feeling relaxed"
]

for sample_input in sample_inputs:
    recommendation = generate_recommendation(sample_input)
    print(f"Prompt: {sample_input} \nRecommended Dish: {recommendation}\n")

Prompt: I want a dish for when I am feeling happy 
Recommended Dish: Dish: Pani Puri

Prompt: I want a dish for when I am feeling sad 
Recommended Dish: Dish: Gongura Pachadi

Prompt: I want a dish for when I am feeling stressed 
Recommended Dish: Dish: Neer Dosa

Prompt: I want a dish for when I am feeling relaxed 
Recommended Dish: Dish: Masor Tenga

